In [1]:
import pandas as pd
from collections import Counter
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
pd.options.display.max_rows=None

In [18]:
file_path = "../data/en.openfoodfacts.org.products.csv.gz"

In [19]:
chunk_size = 10000

filtered_chunks = []

for chunk in pd.read_csv(
    file_path,
    chunksize=chunk_size,
    compression="gzip",
    sep="\t",
    engine="python",
    quoting=3,
):
    filtered_chunks_en = chunk[
        (chunk["countries_tags"] == "en:france") & (chunk["ingredients_tags"].notnull() & chunk["product_name"].notnull())
    ]
    filtered_chunks.append(filtered_chunks_en)

if filtered_chunks:
    data_en = pd.concat(filtered_chunks, axis=0)

In [20]:
# Find the different state of each rows
states_unique_values = set()
data_en["states_tags"].str.split(",").apply(states_unique_values.update)

states_df = pd.DataFrame(data=states_unique_values, columns=["states"])
display(states_df)

,states
0,en:ingredients-photo-to-be-selected
1,en:ingredients-completed
2,en:photos-validated
3,en:categories-completed
4,en:photos-uploaded
5,en:ingredients-photo-selected
6,en:to-be-checked
7,en:photos-to-be-uploaded
8,en:brands-completed
9,en:ingredients-to-be-completed


In [21]:
# Target sta
target_states = [
    "en:completed",
    "en:origins-completed",
    "en:ingredients-completed",
    "en:characteristics-completed",
]

# Construire une expression régulière pour les états
pattern = "|".join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = data_en["states_tags"].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)

data_en_filtered = data_en[data_en["states_tags"].str.contains(pattern)]

Nombre de lignes avec les états spécifiés: 255537


In [22]:
len(data_en) - 258361

1181

In [23]:
describe_sample = data_en_filtered.describe(include="all").drop("top").T

In [24]:
describe_sample.T

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,last_modified_by,last_updated_t,last_updated_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_en,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutrition_data,additives_n,additives,additives_tags,additives_en,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,food_groups,food_groups_tags,food_groups_en,states,states_tags,states_en,brand_owner,ecoscore_score,ecoscore_grade,nutrient_levels_tags,product_quantity,owner,data_quality_errors_tags,unique_scans_n,popularity_tags,completeness,last_image_t,last_image_datetime,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,butyric-acid_100g,caproic-acid_100g,caprylic-acid_100g,capric-acid_100g,lauric-acid_100g,myristic-acid_100g,palmitic-acid_100g,stearic-acid_100g,arachidic-acid_100g,behenic-acid_100g,lignoceric-acid_100g,cerotic-acid_100g,montanic-acid_100g,melissic-acid_100g,unsaturated-fat_100g,monounsaturated-fat_100g,omega-9-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,omega-6-fat_100g,alpha-linolenic-acid_100g,eicosapentaenoic-acid_100g,docosahexaenoic-acid_100g,linoleic-acid_100g,arachidonic-acid_100g,gamma-linolenic-acid_100g,dihomo-gamma-linolenic-acid_100g,oleic-acid_100g,elaidic-acid_100g,gondoic-acid_100g,mead-acid_100g,erucic-acid_100g,nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,added-sugars_100g,sucrose_100g,glucose_100g,fructose_100g,lactose_100g,maltose_100g,maltodextrins_100g,starch_100g,polyols_100g,erythritol_100g,fiber_100g,soluble-fiber_100g,insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,added-salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g,sulphate_100g,nitrate_100g,acidity_100g
count,255537,255537,255536,255537.0,255537,255537.0,255537,249517,255477.0,255477,255537,13865,72394,190388,130135,130135,130135,12100,241099,241092,220552,220552,220552,50434,50398,50397,61678,61655,159399,159399,159399,61551,61540,48041,0.0,51396,76756,112432,255537,255537,255537,255536,255537,255535,99877,0.0,59420,76867,75917,64571,62647.0,13222,255537.0,2,130552,130552,190433.0,255522,222182.0,255536,255537,198915,198915,198915,255537,255537,255537,3875,159029.0,255530,192600,188947.0,50071,8343,186866.0,183512,255537.0,249827.0,249827,220552,220552,237764,237764,222736,222736,181955,181955,84818.0,211997.0,229760.0,27.0,229518.0,227845.0,

In [25]:
threshold = 0.85
fill_threshold_percent = describe_sample[describe_sample["count"] > len(data_en_filtered) * threshold].T.columns
fill_threshold_percent

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime', 'product_name', 'brands',
       'brands_tags', 'categories', 'categories_tags', 'categories_en',
       'countries', 'countries_tags', 'countries_en', 'ingredients_text',
       'ingredients_tags', 'ingredients_analysis_tags', 'additives_n',
       'nutriscore_grade', 'nova_group', 'pnns_groups_1', 'pnns_groups_2',
       'states', 'states_tags', 'states_en', 'ecoscore_grade', 'completeness',
       'last_image_t', 'last_image_datetime', 'main_category',
       'main_category_en', 'image_url', 'image_small_url',
       'image_ingredients_url', 'image_ingredients_small_url', 'energy_100g',
       'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'proteins_100g', 'salt_100g', 'sodium_100g',
       'fruits-vegetables-nuts-estimate-from-ingredients_100g'],
      dtype='object'

In [26]:
columns_to_keep = [
    "code",
    "url",
    "last_modified_datetime",
    "product_name",
    "packaging",
    "packaging_tags",
    "categories",
    "categories_tags",
    "labels_en",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "food_groups_tags",
    "main_category",
    "allergens",
    "additives_tags",
    "traces_tags",
    "nutriscore_grade",
    "ecoscore_grade",
    "nutrient_levels_tags",
    "image_url",
    "image_small_url",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "carbohydrates_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
    "fruits-vegetables-nuts-estimate-from-ingredients_100g",
]
data_en_filtered = data_en_filtered[columns_to_keep]

In [27]:
print(len(data_en_filtered))
# Remove deplicates lines from the dataset
data_en_filtered = data_en_filtered.drop_duplicates()
print(len(data_en_filtered))

255537
255059


In [28]:
print(len(data_en_filtered))
# Remove the duplicates code in the dataset
data_en_filtered = data_en_filtered.drop_duplicates(subset="code")
print(len(data_en_filtered))

255059
254765


In [29]:
# Save the data_en_filtered
data_en_filtered.to_csv("../data/filtered_dataset_openfoodfacts_en.csv", index=False)

### Data Cleaning

In [2]:
data_en_filtered = pd.read_csv("../data/filtered_dataset_openfoodfacts_en.csv", dtype={'code': str})

In [3]:
data_en_filtered.head()

,code,url,last_modified_datetime,product_name,packaging,packaging_tags,categories,categories_tags,labels_en,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,image_small_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,2024-10-20T14:02:34Z,Bio inulin,Glass,en:glass,Gemüse,"en:plant-based-foods-and-beverages,en:plant-based-foods,en:fruits-and-vegetables-based-foods,en:vegetables-based-foods,en:vegetables",No lactose,"en:apricot-juice-and-puree,en:fruit,en:prunus-species-fruit,en:apricot,en:apricot-juice,en:apricot-puree,en:water,en:sugar,en:added-sugar,en:disaccharide","en:palm-oil-free,en:vegan,en:vegetarian","en:fruits-and-vegetables,en:vegetables",en:vegetables,NaN,NaN,NaN,a,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low-quantity,en:sugars-in-moderate-quantity,en:salt-in-low-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.200.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,2024-10-04T09:38:47Z,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film en plastique","en:plastic,en:cardboard,fr:boite-en-carton,fr:film-en-plastique","Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures","en:snacks,en:desserts,en:sweet-snacks,en:biscuits-and-cakes,en:cakes,en:madeleines,en:plain-madeleines","Green Dot,No colorings,No palm oil","en:wheat-flour,en:cereal,en:flour,en:wheat,en:cereal-flour,en:sugar,en:added-sugar,en:disaccharide,en:colza-oil,en:oil-and-fat,en:vegetable-oil-and-fat,en:rapeseed-oil,en:free-range-chicken-eggs,en:egg,en:chicken-egg,en:free-range-eggs,en:glucose-fructose-syrup,en:monosaccharide,en:fructose,en:glucose,en:stabiliser,en:raising-agent,en:e331,en:salt,en:wheat-gluten,en:gluten,en:skimmed-milk-powder,en:dairy,en:milk-powder,en:vegetable-fiber,en:fiber,en:natural-flavouring,en:flavouring,en:e422,fr:carbonates-d-ammonium-carbonates-de-sodium","en:palm-oil-free,en:non-vegan,en:vegetarian-status-unknown","en:sugary-snacks,en:biscuits-and-cakes",en:plain-madeleines,"en:eggs,en:gluten,en:milk","en:e331,en:e422,en:e503","en:nuts,en:soybeans",d,c,"en:fat-in-high-quantity,en:saturated-fat-in-moderate-quantity,en:sugars-in-high-quantity,en:salt-in-moderate-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.200.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667
2,15,http://world-en.openfoodfacts.org/product/00000015/madeleines-chocolait-bijou,2024-10-04T09:38:50Z,Madeleines ChocoLait,"Plastique, Carton","en:plastic,en:cardboard","Snacks, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Gâteaux au chocolat, Madeleines, Madeleines au chocolat","en:snacks,en:sweet-snacks,en:biscuits-and-cakes,en:cakes,en:chocolate-cakes,en:madeleines,en:chocolate-madeleines",NaN,"en:wheat-flour,en:cereal,en:flour,en:wheat,en:cereal-flour,en:milk-chocolate,en:chocolate,en:sugar,en:added-sugar,en:disaccharide,en:colza-oil,en:oil-and-fat,en:vegetable-oil-and-fat,en:rapeseed-oil,en:free-range-chicken-eggs,en:egg,en:chicken-egg,en:free-range-eggs,en:glucose-fructose-syrup,en:monosaccharide,en:fructose,en:glucose,en:stabiliser,en:raising-agent,en:e500,en:e331,en:salt,en:wheat-gluten,en:gluten,en:skimmed-milk-powder,en:dairy,en:milk-powder,en:vegetable-fiber,en:fiber,en:natural-flavouring,en:flavouring,en:cocoa-butter,en:plant,en:cocoa,en:whole-milk-powder,en:cocoa-paste,en:emulsifier,en:natural-vanilla-flavouring,en:vanilla-flavouring,en:e422,en:e503,en:soya-lecithin,en:e322,en:e322i","

In [4]:
def catetegorial_and_numerical_features(df):
    categorical_features = df.select_dtypes(include=['object']).columns
    numerical_features = df.select_dtypes(include=['number']).columns
    return categorical_features, numerical_features

In [5]:
categorical_feature, numerical_feature = catetegorial_and_numerical_features(data_en_filtered)

In [6]:
categorical_feature

Index(['code', 'url', 'last_modified_datetime', 'product_name', 'packaging',
       'packaging_tags', 'categories', 'categories_tags', 'labels_en',
       'ingredients_tags', 'ingredients_analysis_tags', 'food_groups_tags',
       'main_category', 'allergens', 'additives_tags', 'traces_tags',
       'nutriscore_grade', 'ecoscore_grade', 'nutrient_levels_tags',
       'image_url', 'image_small_url'],
      dtype='object')

In [7]:
to_split = [
    "packaging_tags",
    "categories_tags",
    "ingredients_tags",
    "ingredients_analysis_tags",
    "allergens",
    "additives_tags",
    "traces_tags",
    "main_category",
    "food_groups_tags",
]

In [8]:
def clean_dataset(df):
    # Convertir les colonnes de date en années et gérer les erreurs
    df['last_modified_year'] = pd.to_datetime(df['last_modified_datetime'], errors='coerce').dt.year
    df.drop(['last_modified_datetime'], axis=1, inplace=True)

    # Liste des colonnes à remplir avec une valeur par défaut
    fill_unknown = [
        "packaging_tags",
        "categories_tags",
        "ingredients_tags",
        "ingredients_analysis_tags",
        "nutriscore_grade",
        "ecoscore_grade",
        "nutrient_levels_tags",
    ]
    fill_none = [
        "allergens",
        "traces_tags",
        "additives_tags",
        "food_groups_tags",
        "main_category",
        "image_url",
    ]

    # Remplacer les valeurs manquantes
    df[fill_unknown] = df[fill_unknown].fillna("unknown")
    df[fill_none] = df[fill_none].fillna("none")

    return df


# Appliquer la fonction
data_en_filtered_clean = clean_dataset(data_en_filtered)

In [9]:
for feature in to_split:
    data_en_filtered_clean[feature] = data_en_filtered_clean[feature].fillna("").str.replace("en:", "").str.split(",")

In [10]:
def select_frequent_tags(data, column_name, min_count=100, min_for_various_tags=100):
    all_tags = [item for items in data[column_name].dropna() for item in items]
    tag_counts = Counter(all_tags)
    frequent_tags = [tag for tag, count in tag_counts.items() if count >= min_count]
    if len(frequent_tags) > min_for_various_tags:
        frequent_tags = [tag for tag, _ in tag_counts.most_common(min_for_various_tags)]
    frequent_tags.append("other")
    return frequent_tags

In [11]:
# Applique cette fonction aux colonnes de to_split
frequent_tags = {
    col: select_frequent_tags(data_en_filtered_clean, col) for col in to_split
}

In [12]:
frequent_tags.keys() 


dict_keys(['packaging_tags', 'categories_tags', 'ingredients_tags', 'ingredients_analysis_tags', 'allergens', 'additives_tags', 'traces_tags', 'main_category', 'food_groups_tags'])

In [13]:
data_en_filtered_clean["food_groups_tags"].value_counts().reset_index()

,food_groups_tags,count
0,[none],56514
1,"[sugary-snacks, biscuits-and-cakes]",17164
2,"[sugary-snacks, sweets]",16469
3,"[composite-foods, one-dish-meals]",15040
4,"[fish-meat-eggs, processed-meat]",10615
5,"[beverages, unsweetened-beverages]",9974
6,"[cereals-and-potatoes, cereals]",9819
7,"[milk-and-dairy-products, cheese]",8619
8,"[fats-and-sauces, dressings-and-sauces]",8540
9,"[beverages, sweetened-beverages]",7176


In [14]:
len(frequent_tags["food_groups_tags"])

51

In [15]:
#  Transformation en colonnes booléennes seulement pour les tags fréquents
new_columns = {}

for col, tags in frequent_tags.items():
    for tag in tags:
        new_columns[f"{col}_{tag}"] = data_en_filtered[col].apply(
            lambda x: tag in x if isinstance(x, list) else False
        )

    # Créer une colonne booléenne pour les tags non fréquents
    new_columns[f"{col}_other"] = data_en_filtered[col].apply(
        lambda x: any(item not in tags for item in x) if isinstance(x, list) else False
    )

# Convert the dictionary to a DataFrame
new_columns_df = pd.DataFrame(new_columns)

In [28]:
print(data_en_filtered.shape)

print(new_columns_df.shape)

(254765, 29)
(254765, 608)


In [24]:
frequent_tags_list = list(frequent_tags.keys())

In [16]:
data_en_filtered_clean.columns

Index(['code', 'url', 'product_name', 'packaging', 'packaging_tags',
       'categories', 'categories_tags', 'labels_en', 'ingredients_tags',
       'ingredients_analysis_tags', 'food_groups_tags', 'main_category',
       'allergens', 'additives_tags', 'traces_tags', 'nutriscore_grade',
       'ecoscore_grade', 'nutrient_levels_tags', 'image_url',
       'image_small_url', 'energy_100g', 'fat_100g', 'saturated-fat_100g',
       'carbohydrates_100g', 'sugars_100g', 'proteins_100g', 'salt_100g',
       'fruits-vegetables-nuts-estimate-from-ingredients_100g',
       'last_modified_year'],
      dtype='object')

In [25]:
frequent_tags_list

['packaging_tags',
 'categories_tags',
 'ingredients_tags',
 'ingredients_analysis_tags',
 'allergens',
 'additives_tags',
 'traces_tags',
 'main_category',
 'food_groups_tags']

In [ ]:
data_en_filtered_clean[
    [
        [
            "packaging_tags",
            "categories_tags",
            "ingredients_tags",
            "ingredients_analysis_tags",
            "allergens",
            "additives_tags",
            "traces_tags",
            "main_category",
            "food_groups_tags",
        ]
    ]
]

,code,url,product_name,packaging,packaging_tags,categories,categories_tags,labels_en,ingredients_tags,ingredients_analysis_tags,food_groups_tags,main_category,allergens,additives_tags,traces_tags,nutriscore_grade,ecoscore_grade,nutrient_levels_tags,image_url,image_small_url,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,last_modified_year
0,5,http://world-en.openfoodfacts.org/product/00000005/bio-inulin-ewl,Bio inulin,Glass,[glass],Gemüse,"[plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, vegetables-based-foods, vegetables]",No lactose,"[apricot-juice-and-puree, fruit, prunus-species-fruit, apricot, apricot-juice, apricot-puree, water, sugar, added-sugar, disaccharide]","[palm-oil-free, vegan, vegetarian]","[fruits-and-vegetables, vegetables]",[vegetables],[none],[none],[none],a,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low-quantity,en:sugars-in-moderate-quantity,en:salt-in-low-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0005/front_de.34.200.jpg,840.0,0.0,0.0,8.0,8.0,0.0,0.23,50.000000,2024
1,10,http://world-en.openfoodfacts.org/product/00000010/madeleines-nature-bijou,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film en plastique","[plastic, cardboard, fr:boite-en-carton, fr:film-en-plastique]","Snacks, Desserts, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Madeleines, Madeleines natures","[snacks, desserts, sweet-snacks, biscuits-and-cakes, cakes, madeleines, plain-madeleines]","Green Dot,No colorings,No palm oil","[wheat-flour, cereal, flour, wheat, cereal-flour, sugar, added-sugar, disaccharide, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, free-range-chicken-eggs, egg, chicken-egg, free-range-eggs, glucose-fructose-syrup, monosaccharide, fructose, glucose, stabiliser, raising-agent, e331, salt, wheat-gluten, gluten, skimmed-milk-powder, dairy, milk-powder, vegetable-fiber, fiber, natural-flavouring, flavouring, e422, fr:carbonates-d-ammonium-carbonates-de-sodium]","[palm-oil-free, non-vegan, vegetarian-status-unknown]","[sugary-snacks, biscuits-and-cakes]",[plain-madeleines],"[eggs, gluten, milk]","[e331, e422, e503]","[nuts, soybeans]",d,c,"en:fat-in-high-quantity,en:saturated-fat-in-moderate-quantity,en:sugars-in-high-quantity,en:salt-in-moderate-quantity",https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.400.jpg,https://images.openfoodfacts.org/images/products/000/000/000/0010/front_fr.37.200.jpg,1852.0,22.0,2.6,54.0,25.0,6.4,0.53,22.666667,2024
2,15,http://world-en.openfoodfacts.org/product/00000015/madeleines-chocolait-bijou,Madeleines ChocoLait,"Plastique, Carton","[plastic, cardboard]","Snacks, Snacks sucrés, Biscuits et gâteaux, Gâteaux, Gâteaux au chocolat, Madeleines, Madeleines au chocolat","[snacks, sweet-snacks, biscuits-and-cakes, cakes, chocolate-cakes, madeleines, chocolate-madeleines]",NaN,"[wheat-flour, cereal, flour, wheat, cereal-flour, milk-chocolate, chocolate, sugar, added-sugar, disaccharide, colza-oil, oil-and-fat, vegetable-oil-and-fat, rapeseed-oil, free-range-chicken-eggs, egg, chicken-egg, free-range-eggs, glucose-fructose-syrup, monosaccharide, fructose, glucose, stabiliser, raising-agent, e500, e331, salt, wheat-gluten, gluten, skimmed-milk-powder, dairy, milk-powder, vegetable-fiber, fiber, natural-flavouring, flavouring, cocoa-butter, plant, cocoa, whole-milk-powder, cocoa-paste, emulsifier, natural-vanilla-flavouring, vanilla-flavouring, e422, e503, soya-lecithin, e322, e322i]","[palm-oil-free, non-vegan, maybe-vegetarian]","[sugary-snacks, biscuits-and-cakes]",[chocolate-madeleines],[none],"[e322, e322i, e331, e422, e500, e503]",[nuts],d,d,"en:fat-in-high-quantity,en:saturated-fat-in-high-quantity,en:sugars-in-high-quantity,en:salt-in-moderate-quantity",https://images.openf

In [11]:
# data_en_filtered = pd.concat([data_en_filtered, new_columns_df], axis=1)

In [ ]:
describe_sample = data_en_filtered.describe(include="all").drop("top").T